# Creating Redshift Cluster

### This notebook provides the tools to create, delete and query a redshift cluster for the data warehouse project.

## Load DWH Params from a configuration file

In [ ]:
import pandas as pd
import boto3
import json
import getpass
import psycopg2
import  configparser as parser
from botocore.exceptions import ClientError

### AWS CONFIGURATION

In [ ]:
# checking Redshift Properties
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', 1)
    keysToShow = ["ClusterIdentifier", "NodeType", 
                  "ClusterStatus", "MasterUsername", "DBName", 
                  "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [ ]:
# Enter AWS KEY and Secret
KEY = getpass.getpass(prompt='Enter AWS Access key ID:')
SECRET = getpass.getpass(prompt='Enter AWS Secret access key')

In [ ]:
config = parser.ConfigParser()

config.read_file(open('dwh.cfg'))

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

pd.DataFrame({
        "Params": [
                "DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DB_NAME", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT","DWH_IAM_ROLE_NAME", "KEY", "SECRET"
            ],
        "Value": [
            DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME, KEY, SECRET
        ]
    })

## Create clients for IAM and Redshift

In [ ]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )
iam = boto3.client('iam',
                   region_name="us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET,
                   )
redshift = boto3.client('redshift',
                        region_name="us-west-2",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

In [ ]:
# Check out the sample data sources on S3
sampleDbBucket =  s3.Bucket("udacity-dend")

for obj in sampleDbBucket.objects.filter(Prefix="log_data"):
    print(obj)

## Create IAM role and Redshift cluster

### 1. Create IAM Role

In [ ]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

### Creating a Redshift cluster

In [ ]:
# Initializing a Redshift cluster

try:
    response = redshift.create_cluster(
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,

        #Roles (for s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [ ]:
# checking Redshift Properties
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', 1)
    keysToShow = ["ClusterIdentifier", "NodeType",
                  "ClusterStatus", "MasterUsername", "DBName",
                  "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
        return pd.DataFrame(data=x, columns=["Key", "Value"])

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

#### Checking current status of previous cluster

In [ ]:
clusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
DWH_ENDPOINT = clusterProps['Endpoint']['Address']
prettyRedshiftProps(clusterProps)

#### Getting the endpoint and role ARN

In [ ]:
DWH_ENDPOINT = clusterProps['Endpoint']['Address']
DWH_ROLE_ARN = clusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

In [ ]:
# Export dwh configuration file

config_dwh = parser.ConfigParser()


config_dwh['CLUSTER'] = {
    'HOST' : DWH_ENDPOINT,
    'DB_NAME' : config['DWH']['DWH_DB'],
    'DB_USER' : config['DWH']['DWH_DB_USER'],
    'DB_PASSWORD' : config['DWH']['DWH_DB_PASSWORD'],
    'DB_PORT' : config['DWH']['DWH_PORT']
}

config_dwh['IAM_ROLE'] = {'ARN' : DWH_ROLE_ARN}
config_dwh['S3'] = config['S3']

with open('dwh.cfg', 'w') as config_dwh_file:
    config_dwh.write(config_dwh_file)

## Check connection of the cluster

## STEP 3: Open an incoming  TCP port to access the cluster endpoint


In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

#### Run queries on the Redshift cluster

In [ ]:
# Run a query
%reload_ext sql

In [ ]:
from sqlalchemy import create_engine, text

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

try:
    with engine.connect() as connection:
        result = connection.execute("SELECT 1;")
        print(conn_string)
        print("Connect successfully!")
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
# Establish connection to the cluster
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
%sql $conn_string

In [ ]:
%sql SELECT DISTINCT tablename FROM pg_table_def WHERE schemaname = 'public';

In [ ]:
!python3 create_tables.py

In [ ]:
!python3 etl.py

In [ ]:
from sqlalchemy import create_engine

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

connection = engine.connect()

query = 'SELECT * FROM time limit 10;'

result = connection.execute(query).fetchall()
print(result)
for e in result:
    print(e)


In [ ]:
from sqlalchemy import create_engine

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

conn = engine.connect()
query = 'SELECT * FROM time limit 10;'
sql_query = pd.read_sql(query, conn)
df = pd.DataFrame(sql_query, columns = ['start_time', 'hour', 'day','week','month', 'year', 'weekday'])

print(df)


In [ ]:
from sqlalchemy import create_engine

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

conn = engine.connect()
query = 'SELECT * FROM artists limit 10;'
sql_query = pd.read_sql(query, conn)
df = pd.DataFrame(sql_query, columns = ['artist_id', 'name', 'location', 'latitude', 'longitude'])

print(df)

In [ ]:
from sqlalchemy import create_engine

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

conn = engine.connect()
query = 'SELECT * FROM staging_songs WHERE artist_name IS NOT NULL;'
sql_query = pd.read_sql(query, conn)
df = pd.DataFrame(sql_query)

print(df)

In [ ]:
from sqlalchemy import create_engine

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
engine = create_engine(conn_string)

conn = engine.connect()
query = 'SELECT * FROM songplays order by songplay_id limit 10;'
sql_query = pd.read_sql(query, conn)
df = pd.DataFrame(sql_query, columns = ['start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent'])

print(df)

## Clean up the resource

In [ ]:
# Deleting the cluster
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
# Checking current status of cluster
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!